In [85]:
import pandas as pd
import os
import glob

import mikeio1d
from mikeio1d import res1d
from mikeio1d.res1d import Res1D, QueryData, QueryDataReach

# Reading the input csv file containing item name, khal name and chainage
input_file_path = r"G:\MJI\SKZ\Script\MIKE 1D Result Extraction\Input_file_2.csv"
item_df = pd.read_csv(input_file_path)

# Reading the mike11 result file
res_filename = r"G:\MJI\SKZ\Script\MIKE 1D Result Extraction\Result_file\P_59_2_existing_CC_2050_RCP85.res11"


# Defining an blank dataframe to store the khal water level
m_df = pd.DataFrame()

# Iterating through the result file to extract the required khal water level
for item,branch,chainage in zip(item_df['Item_type'],item_df['Branch_Name'],item_df['Chainage']):
    
    # Defining branch name with chainage
    branch_reach_name = '{} {}'.format(branch,str(chainage))
    
    chainage = float(chainage)
    # Reading the mike11 result file and extracting required chainage water level data
    query = QueryDataReach(item, branch, chainage)
    r_df = Res1D(res_filename).read(query)
    
    # Renaming the extract water level data column
    r_df.rename(columns = {str(query):branch_reach_name}, inplace = True)
    
    # Updating the extracted water data in the water level dataframe
    if m_df.size == 0:
        m_df = pd.concat([m_df, r_df])
        
    else:
        m_df = m_df.merge(r_df, left_index=True, right_index=True)

    print(branch_reach_name)

    
### DETERMINING MAXIMUM WATER LEVEL FROM 3-DAYS ROLLING MINIMUM WATER LEVEL

# Determing 3-days moving minimum water level from extracted water level data
# Since model result is stored at 30 mins interval so there are 48 timestep daily and 144 in 3-days
# And also not considering the first three days of water to ignore the model instability
c_df = m_df[144:].rolling(window=144).min()

# Determing the maximum water level at each chainage of khals
n_df = c_df[:].max()

# Converting chainage wise khal maximum water level to pandas dataframe
# and renaming the series column as 'WL'
p_df = n_df.to_frame()
p_df.columns = ['WL_mPWD']

# Inserting the khal and chainge column at '0' index of the dataframe using the index value
p_df.insert(loc=0, column ='Khal_Chainage', value = n_df.index)

# Resetting the index column of the dataframe and dropping the duplicate columns
p_df.reset_index(drop =True, inplace = True)
p_df.drop_duplicates(keep='first', inplace=True)

# Splitting the 'Khal_Chainage' column and creating khal name and corresponding chainage column
# and converting the chainage column from string type and float type
p_df[['Khal_Name', 'Chainage']] = p_df['Khal_Chainage'].str.split(' ',expand=True)
p_df['Chainage'] = p_df['Chainage'].astype(float)

K_KHAL_1 0.0
K_KHAL_1 501.0
K_KHAL_1 1007.0
K_KHAL_1 1511.0
K_KHAL_1 2018.0
K_KHAL_1 2345.0
K_KHAL_5 0.0
K_KHAL_5 511.0
K_KHAL_5 1010.0
K_KHAL_5 1514.0
K_KHAL_5 1806.0
K_KHAL_4 0.0
K_KHAL_4 420.0
K_KHAL_8 0.0
K_KHAL_8 270.0
K_KHAL_8 506.094
K_KHAL_8 774.0
K_KHAL_8 1254.0
K_KHAL_8 1760.0
K_KHAL_8 2214.0
K_KHAL_8 2263.0
K_KHAL_10 0.0
K_KHAL_10 496.0
K_KHAL_10 1066.0
K_KHAL_10 1548.0
K_KHAL_10 1748.0
K_KHAL_11 0.0
K_KHAL_11 382.0
K_KHAL_11 477.0
K_KHAL_12 0.0
K_KHAL_12 353.474
K_KHAL_12 576.0
K_KHAL_12 1047.0
K_KHAL_12 1469.0
K_KHAL_12 1976.0
K_KHAL_12 2184.0
K_KHAL_12 2520.0
K_KHAL_13 0.0
K_KHAL_13 297.0
K_KHAL_13 801.0
K_KHAL_13 1306.0
K_KHAL_13 1810.0
K_KHAL_13 2202.0
K_KHAL_13 2310.0
K_KHAL_13 2451.0
K_KHAL_13 2823.0
K_KHAL_14 0.0
K_KHAL_14 560.0
K_KHAL_14 997.0
K_KHAL_14 1064.0
K_KHAL_16 0.0
K_KHAL_16 941.0
K_KHAL_16 1439.0
K_KHAL_16 1984.0
K_KHAL_16 2450.0
K_KHAL_16 2963.0
K_KHAL_16 3090.0
K_KHAL_16 3202.0
K_KHAL_16 3455.0
K_KHAL_18 0.0
K_KHAL_18 661.0
K_KHAL_19 0.0
K_KHAL_19 482.0


In [87]:
import pandas as pd
import numpy as np
import glob
import os

# Creating a function to remove the spaces from each item of a string list and returing as a space free item list
def string_space_removal(cons_list):
    
    cons_name_list = []
    
    for i in range(0, len(cons_list), 1):
        
        cons_name = cons_list[i].strip() #[Remove space at the beginning and at the end of the string]
    
        cons_name_list.append(cons_name)
    
    return cons_name_list

# Reading the co-odinate csv file and selecting the required rows
rc_df = pd.read_csv(r"G:\MJI\SKZ\Script\MIKE 1D Result Extraction\P_59_2_existing_CC_2050_RCP85.csv")
selected_df = rc_df[16:-1]

# Extracting the x_coordinate
x_coord = selected_df['MIKE 11 Result File Information'].str[2:12]

# Extracting the y_coordinate
y_coord = selected_df['MIKE 11 Result File Information'].str[14:24]

# Getting the khal/River name
river_start_index = 27
max_river_character = 17
river_end_index = river_start_index + max_river_character

river = list(selected_df['MIKE 11 Result File Information'].str[river_start_index:river_end_index])
river_name = string_space_removal(river)

# Extracting the corresponding khal chainage
chainage_id = list(selected_df['MIKE 11 Result File Information'].str[river_end_index+2:river_end_index+2+10])
chainage = string_space_removal(chainage_id)

# Creating the coordinate dataframe from the extracted data
coord_df = pd.DataFrame({'BTM_X': map(float, x_coord), 'BTM_Y': map(float, y_coord), 
                         'Khal_Name':map(str, river_name), 'Chainage':map(float, chainage_id)})

# Removing the duplicate columns from the dataframe
coord_df.drop_duplicates(keep='first', inplace=True)

In [88]:
# Conbining the maximum of 3-days rolling minimum and co-ordinate dataframe
f_df = pd.merge(p_df,coord_df, how='inner', on = ['Khal_Name', 'Chainage'])

In [94]:
# Subsetting the required column from the dataframe
out_df = f_df[['Khal_Name', 'Chainage', 'BTM_X', 'BTM_Y', 'WL_mPWD']]

# Saving the khal chainage wise maximum of 3-days rolling minimum water level dataframe
out_df.to_excel(r'final_test.xlsx', index = False, float_format = '%.3f')